# Node prioritisation in Leukemia with regulatory and metabolic networks

In this tutorial, we illustrate how MultiXrank can be used to prioritise genes and drugs of interest in leukemia. 
We build upon a multilayer network composed of:
    
- a gene multiplex network, encoding proteins associations in protein-protrin interactions, protein complexes and pathways; 
- a drug multiplex network, encoding clinically reported drug-drug interactions, experimental and predicted drug combinations, and phramacological interactions.

In this tutorial, we complete this multilayer network with two additional types of interactions:
	
- **Network of metabolic reactions**: We dowloaded the human genome-scale metabolic model from the Human-GEM database and create a network of metabolic reactions.  Specifically, this additional layer contains nodes representing specific human metabolic reactions. Directed edges connect two reactions when the product of one reaction corresponds to the reactants of another. We connect the metabolic reaction network to the gene multiplex through bipartite reactions for each gene known to participate in a metabolic reaction.	
- **Regulatory network of Transcription Factors and their targets**: We add to the existing gene multiplex a new layer featuring directed interactions representing Transcription Factors (TF) and their regulated targets in leukemia. This network was constructed using data from the GRNdb database.

Finally, we run MultiXrank on this novel multilayer netork to prioritise genes and drugs of interest in leukemia, using the HRAS gene and the Tipifarnib drug as seeds.

# 1) Network of metabolic reactions

The human-GEM was downaloaded from: https://github.com/SysBioChalmers/Human-GEM

    J. L. Robinson, P. Kocabas, H. Wang, P.-E. Cholley, et al. An atlas of human metabolism. Sci. Signal. 13, eaaz1482 (2020). doi:10.1126/scisignal.aaz1482



In [1]:
import pandas as pd 
import networkx as nx
import re

In [2]:
data = pd.read_csv("Human-GEM.txt", sep="\t")
data

,Rxn name,Formula,Gene-reaction association,LB,UB,Objective
0,MAR03905,MAM01796c + MAM02552c -> MAM01249c + MAM02039...,ENSG00000147576 or ENSG00000172955 or ENSG0000...,0.0,1000.0,0.0
1,MAR03907,MAM01796c + MAM02554c -> MAM01249c + MAM02039...,ENSG00000117448,0.0,1000.0,0.0
2,MAR04097,MAM01252c + MAM01371c + MAM01597c -> MAM01261...,ENSG00000131069,0.0,1000.0,0.0
3,MAR04099,MAM01252m + MAM01371m + MAM01597m -> MAM01261...,ENSG00000111058 or ENSG00000154930,0.0,1000.0,0.0
4,MAR04108,MAM01257c + MAM01597c -> MAM01261c + MAM01334...,ENSG00000131069,0.0,1000.0,0.0
...,...,...,...,...,...,...
12990,MAR20168,MAM00209c -> MAM02393m,ENSG00000144182,0.0,1000.0,0.0
12991,MAR20169,MAM01412c + MAM02040c -> MAM01410c + MAM01597...,ENSG00000172497,0.0,1000.0,0.0
12992,MAR20170,MAM02040c + MAM02644c -> MAM01597c + MAM02039...,ENSG00000097021,0.0,1000.0,0.0
12993,MAR20171,MAM01650c + MAM02040c -> MAM01597c + MAM01648...,ENSG00000097021,0.0,1000.0,0.0


## Create a dictionary of reactions reactants and products

In [3]:
separators = re.compile(r'\s*(->|<=>)\s*')

mar_reactants_dict = {}
mar_products_dict = {}

for index, row in data.iterrows():
    rxn_id = row['Rxn name']
    
    reactants, sep, products = re.split(separators, row['Formula'])
    
    mar_reactants_dict[rxn_id] = reactants.split(' + ')
    
    mar_products_dict[rxn_id] = products.split(' + ')



## Create the network of metabolic reactions

Directed edge list of reactions: a directed edge is created between two reactions when the product of the first reaction is used in the reactants of the second reaction.

In [4]:
edge_list = []

# Iterate through reaction IDs
for reaction_id2, products2 in mar_products_dict.items():
    for reaction_id1, reactants1 in mar_reactants_dict.items():
        common_metabolites = set(reactants1) & set(products2)
        if common_metabolites:
            edge_list.append((reaction_id2, reaction_id1, list(common_metabolites)))


In [5]:
edges = pd.DataFrame(edge_list, columns=['Reaction1', 'Reaction2', 'CommonMetabolites'])

edges[['Reaction1', 'Reaction2']].to_csv('multiplex/3/mar_mar.tsv', sep='\t', index=False, header=False)


## Create a dictionary of gene-reaction associations

In [6]:
mar_gene_dict = {}
for index, row in data.iterrows():
    rxn_id = row['Rxn name']
    
    if isinstance(row['Gene-reaction association'], str):
        # Extract all strings starting with ENSG and ending with a space
        genes = re.findall(r'\bENSG[0-9]+\b', row['Gene-reaction association'])
        
        mar_gene_dict[rxn_id] = set(genes)

Convert ensembl ids to gene symbol

In [7]:
genes = pd.read_csv("genes.tsv", sep="\t") # Downloaded from Human-GEM
genes

,genes,geneENSTID,geneENSPID,geneUniProtID,geneSymbols,geneEntrezID,geneNames,geneAliases,compartments,compDataSource
0,ENSG00000000419,ENST00000466152;ENST00000371582;ENST0000068304...,ENSP00000507119;ENSP00000360638;ENSP0000050698...,O60762,DPM1,8813,dolichyl-phosphate mannosyltransferase subunit...,CDGIE;MPDS,Endoplasmic reticulum,SwissProt
1,ENSG00000001036,ENST00000002165;ENST00000451668;ENST00000367585,ENSP00000002165;ENSP00000398119,Q9BTY2,FUCA2,2519,alpha-L-fucosidase 2,dJ20N2.5;MGC1314,Extracellular,DeepLoc2
2,ENSG00000001084,ENST00000509541;ENST00000650454;ENST0000061692...,ENSP00000495056;ENSP00000497574;ENSP0000048275...,P48506,GCLC,2729,glutamate-cysteine ligase catalytic subunit,GCS;GLCL;GLCLC,Cytosol;Nucleus,CellAtlas
3,ENSG00000001630,ENST00000003100;ENST00000450723;ENST0000042286...,ENSP00000003100;ENSP00000406757;ENSP00000394268,Q16850,CYP51A1,1595,cytochrome P450 family 51 subfamily A member 1,CP51;CYP51;CYPL1;LDM;P450-14DM;P450L1,Endoplasmic reticulum,SwissProt;CellAtlas
4,ENSG00000002549,ENST00000226299;ENST00000618908;ENST0000050849...,ENSP00000226299;ENSP00000481000;ENSP0000047602...,P28838,LAP3,51056,leucine aminopeptidase 3,LAP;LAPEP;PEPS,Nucleus;Cytosol,SwissProt;CellAtlas
...,...,...,...,...,...,...,...,...,...,...
2884,ENSG00000114120,ENST00000324194;ENST00000446041;ENST0000050742...,ENSP00000320688;ENSP00000401938;ENSP0000042147...,Q96CQ1,SLC25A36,55186,solute carrier family 25 member 36,NaN,NaN,NaN
2885,ENSG00000159445,ENST00000368814.8;ENST00000489410.1;ENST000004...,ENSP00000357804.3;ENSP00000433304.1;ENSP000004...,Q5T1C6,THEM4,117145,thioesterase superfamily member 4,CTMP,Cytosol;Inner mitochondria,SwissProt
2886,ENSG00000196407,ENST00000368817.10;ENST00000453881.2,ENSP00000357807.4;ENSP00000406809.2,Q8N1Q8,THEM5,284486,thioesterase superfamily member 5,ACOT15,Inner mitochondria,SwissProt
2887,ENSG00000013306,ENST00000377095;ENST00000225308;ENST0000059019...,ENSP00000366299;ENSP00000225308;ENSP0000046768...,Q9BZJ4,SLC25A39,51629,solute carrier family 25 member 39,NaN,NaN,NaN


In [8]:
ensg_to_symbol = dict(zip(genes['genes'], genes['geneSymbols']))

mapped_mar_gene_dict = {mar_id: [ensg_to_symbol.get(ensg, ensg) for ensg in engrn] for mar_id, engrn in mar_gene_dict.items()}


In [9]:
bipartites = [(mar_id, gene) for mar_id, genes in mapped_mar_gene_dict.items() for gene in genes]

bipartites = pd.DataFrame(bipartites, columns=['MAR_ID', 'Gene'])
bipartites

,MAR_ID,Gene
0,MAR03905,ADH6
1,MAR03905,ADH1A
2,MAR03905,ADH7
3,MAR03905,ADH1C
4,MAR03905,ADH5
...,...,...
25889,MAR20168,LIPT1
25890,MAR20169,ACOT12
25891,MAR20170,ACOT7
25892,MAR20171,ACOT7


In [10]:
bipartites.to_csv('bipartite/3_1.tsv', sep='\t', index=False, header=False)


# 2) Network of Transcription Factor - target interactions

The data was downloaded from http://www.grndb.com/

    Fang et al. GRNdb: decoding the gene regulatory networks in diverse human and mouse conditions, Nucleic Acids Research, 2020 (DOI: 10.1093/nar/gkaa995)

    Li et al. Single-cell transcriptomic analysis reveals dynamic alternative splicing and gene regulatory networks among pancreatic islets, Science China Life Sciences, 2020 (DOI: 10.1007/s11427-020-1711-x)

In [11]:
tf = pd.read_csv('AML_TCGA-regulons.txt', sep="\t")
tf

,TF,gene,bestMotif,NES,Genie3Weight,Confidence
0,A1CF,GNAT1,hdpi__ACF,3.45,0.007303,High
1,A1CF,RPL23AP87,hdpi__ACF,3.45,0.008207,High
2,AHCTF1,AHCTF1,cisbp__M0131,3.03,NaN,High
3,AHCTF1,AP1G1,cisbp__M0131,3.03,0.009322,High
4,AHCTF1,DYRK1A,cisbp__M0131,3.03,0.014870,High
...,...,...,...,...,...,...
115375,ZXDC,TECPR2,cisbp__M0507,3.09,0.011065,High
115376,ZXDC,TMEM110,cisbp__M0507,3.09,0.008685,High
115377,ZXDC,TRAK1,cisbp__M0507,3.09,0.008408,High
115378,ZXDC,TRIM72,cisbp__M0507,3.09,0.003858,High


In [12]:
tf[["TF", "gene"]].to_csv("multiplex/1/tf_target.tsv", sep="\t", index=False, header=False)

# 3)  Running MultiXrank

### 3.1) Creating the seed file

To indicate the set of seeds to be used in the Random Walk with Restart, we create a ```seeds.txt``` file containing the ids of our two nodes of interest:
```
    HRAS
    DB04960
```

### 3.2) Creating the configuration file

We also have to create the configuration file, ```config_full.yml``` that holds the parameters to be used when running MultiXrank. The documentation to create this configuration file is available here: https://multixrank-doc.readthedocs.io/en/latest/content/reference.html#configuration-and-network-files

```
seed: seeds.txt
r: 0.7 # Restart probability
# self_loops: Are self-loops allowed? 1 yes, 0 not
self_loops: 0
# eta: Restart probability for each multiplex. Must sum up to one.
eta: [1/3,1/3, 1/3]
#4. Inter multiplex Networks Jump Probability (lambda)
lamb:
    - [1/3, 1/3, 1/3]
    - [1/3, 1/3, 1/3]
    - [1/3, 1/3, 1/3]
multiplex:
    1:
        layers:
            - multiplex/1/Complexes.tsv
            - multiplex/1/PPI.tsv
            - multiplex/1/Reactome.tsv
            - multiplex/1/tf_target.tsv # This layer is directed (graph_type = 10)
        delta: 0.5
        # Graph type: undirected/directed, unweighted/weighted
        graph_type: [00, 00, 00, 10]
        # Restart probability. Layers Restart Probability (tau)
        tau: [1/4, 1/4, 1/4, 1/4]
    2:
        layers:
            - multiplex/2/clinical_drug_interactions.tsv
            - multiplex/2/drugbank-chem-chem.tsv
            - multiplex/2/experimental_drug_combinations.tsv
            - multiplex/2/predicted_drug_combinations.tsv
        delta: 0.5
        graph_type: [00, 00, 00, 00]
        tau: [1/4, 1/4, 1/4, 1/4]
    3:
        layers:
            - multiplex/3/mar_mar.tsv # This layer is directed (graph_type = 10)
        delta: 0.5
        graph_type: [10]
        tau: [1]
bipartite:
    bipartite/2_1.tsv: {source: 2, 'target': 1, graph_type: 00}
    bipartite/3_1.tsv: {source: 3, 'target': 1, graph_type: 00} # The novel bipartite network connecting proteins and metabolic reactions

```

### 3.3) Running MultiXrank

In [13]:
import multixrank

In [14]:
# Create MultiXrank object, using the configuration file
multixrank_obj = multixrank.Multixrank(config="config_full.yml", wdir='./')

# Perform the Random Walk with Restart
ranking_df = multixrank_obj.random_walk_rank()

# Save the results
multixrank_obj.write_ranking(ranking_df, path="multiXrank_results/", top=None)


# 4)  Exploring MultiXrank results

### 4.1) Prioritised genes

The results of the node prioritisation for the gene multiplex are output in file `multiXrank_results/multiplex_1.tsv`. 

In [18]:
gene_prio = pd.read_csv('multiXrank_results/multiplex_1.tsv', sep="\t")
gene_prio.head(11)

,multiplex,node,score
0,1,HRAS,0.091300
1,1,CYP3A4,0.005630
2,1,FNTB,0.005460
3,1,ZBTB17,0.001017
4,1,RASGRP1,0.000431
5,1,RAF1,0.000310
6,1,GABPA,0.000297
7,1,RIN1,0.000285
8,1,DGKZ,0.000261
9,1,AURKA,0.000088


### 4.2) Prioritised drugs 

The results of the node prioritisation for the gene multiplex are output in file `multiXrank_results/multiplex_2.tsv`. 

In [20]:
drug_prio = pd.read_csv('multiXrank_results/multiplex_2.tsv', sep="\t")
drug_prio.head(11)

,multiplex,node,score
0,2,DB04960,0.104663
1,2,DB00399,0.000335
2,2,DB00773,0.000330
3,2,DB00398,0.000225
4,2,DB00637,0.000166
5,2,DB01380,0.000164
6,2,DB00630,0.000150
7,2,DB01254,0.000086
8,2,DB01268,0.000084
9,2,DB06589,0.000053


### 4.3) Prioritised metabolic reactions 

Because we have added a novel layer for metabolic reactions, we are now able to prioritise metabolic reactions associated with our seeds. The results of the node prioritisation for the metabolic reaction monoplex are output in file `multiXrank_results/multiplex_3.tsv`. 

In [21]:
reac_prio = pd.read_csv('multiXrank_results/multiplex_3.tsv', sep="\t")
reac_prio.head(11)

,multiplex,node,score
0,3,MAR07166,0.002210
1,3,MAR00604,0.000236
2,3,MAR00141,0.000235
3,3,MAR01460,0.000152
4,3,MAR09499,0.000148
5,3,MAR06555,0.000148
6,3,MAR07165,0.000147
7,3,MAR09491,0.000107
8,3,MAR08816,0.000086
9,3,MAR00158,0.000042


### 4.4) Comments on the results

The findings in our novel results align with those obtained in our initial application:

* The same set of drugs were prioritised in both applications, although with minor changes in their ranking. For instance, Zoledronic acid (DB00399), previously ranked top 5, now holds the first position.

* Eight out of the top 10 ranked genes from the previous results were also prioritised in the new results, with slight variations in their ranking. Notably, two novel genes, ZBTB17 and GABPA, were prioritised, ranking 3rd and 6th, respectively. Interestingly, ZBTB17 codes for a protein involved in the regulation of c-myc, which is known to play a crucial role in leukogenesis (1).  GABPA encodes a subunit of the GA-binding protein transcription factor, implicated in chronic myelogenous leukemia development (2).

The incorporation of the metabolic network as a new layer in the multilayer network allowed us to prioritize metabolic reactions. The top-ranked reactions were linked to Terpenoid backbone biosynthesis, Acylglycerides metabolism, Glycerophospholipid metabolism, Cholesterol metabolism, and Inositol phosphate metabolism. Notably, Cholesterol metabolism and Inositol phosphate metabolism have been documented as being altered in leukemia (3,4). More broadly, disrupted lipid metabolism is increasingly recognized as a hallmark of cancer (5).


    (1) Salvatori B, Iosue I, Djodji Damas N, Mangiavacchi A, Chiaretti S, Messina M, Padula F, Guarini A, Bozzoni I, Fazi F, Fatica A. Critical Role of c-Myc in Acute Myeloid Leukemia Involving Direct Regulation of miR-26a and Histone Methyltransferase EZH2. Genes Cancer. 2011 May;2(5):585-92. doi: 10.1177/1947601911416357. PMID: 21901171; PMCID: PMC3161419.
    
    (2) Yang, Z. F., Zhang, H., Ma, L., Peng, C., Chen, Y., Wang, J., ... & Rosmarin, A. G. (2013). GABP transcription factor is required for development of chronic myelogenous leukemia via its control of PRKD2. Proceedings of the National Academy of Sciences, 110(6), 2312-2317.
    
    (3) Zhao, L., Zhan, H., Jiang, X., Li, Y., & Zeng, H. (2019). The role of cholesterol metabolism in leukemia. Blood Science, 1(01), 44-49.
    
    (4) Bagacean, C., Iuga, C. A., Bordron, A., Tempescul, A., Pralea, I. E., Bernard, D., ... & Renaudineau, Y. (2022). Identification of altered cell signaling pathways using proteomic profiling in stable and progressive chronic lymphocytic leukemia. Journal of Leukocyte Biology, 111(2), 313-325.
    
    (5) Karagiota, A., Chachami, G., & Paraskeva, E. (2022). Lipid metabolism in cancer: The role of acylglycerolphosphate acyltransferases (AGPATs). Cancers, 14(1), 228.